![](https://github.com/vorodrigues/databricks-automl-lab/blob/main/img/header-automl.png?raw=true)

# Lab 01 - Exploração e Preparação de Dados

## Exercício 01.01 - Exploração dos Dados

O primeiro passo em qualquer jornada analítica é entender nossos dados e identificar ações que podemos tomar para melhorar seu poder preditivo.

Vamos começar criando algumas análises para explorar nossos dados.

### A. Visualizando os dados

Nesse projeto, vamos trabalhar com três tabelas:
- **visits:** transações efetuadas em ATMs
- **customers:** dados cadastrais dos clientes
- **locations:** dados de localização dos ATMs

Vamos ver como são esses dados!

Antes de tudo, vamos selecionar nosso banco de dados.

Preencha o valor da variável `db` e execute a célula abaixo.

In [0]:
db = 'meu_db'
spark.sql(f'USE {db}')

Execute a célula abaixo para visualizar a tabela **visits**

In [0]:
df_visits = spark.table('visits')
display(df_visits)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.

Execute a célula abaixo para visualizar a tabela **customers**

In [0]:
df_customers = spark.table('customers')
display(df_customers)

Execute a célula abaixo para visualizar a tabela **locations**

In [0]:
df_locations = spark.table('locations')
display(df_locations)

### B. Criando visualizações

Agora, vamos criar algumas visualizações para enxergar melhor alguns padrões de interesse nos dados.

Na célula com os dados de transações (visits), execute os seguintes passos:
- Clique no ícone **+**, logo abaixo do código e ao lado de **Table**
- Clique em **Visualization**
- Em **Visualization type**, selecione **Pie**
- Em **X column**, selecione **fraud_report**
- Em **Y columns**, selecione **\***
- Clique em **Save**

Vamos criar mais uma visualização nessa mesma célula:
- Clique no ícone **+**, logo abaixo do código e ao lado de **Table**
- Clique em **Visualization**
- Em **Visualization type**, selecione **Bar**
- Em **X column**, selecione **hour**
- Em **Y columns**, selecione **amount**
- Em **Group by**, selecione **fraud_report**
- Clique em **Save**

### C. Perfilamento de dados

Usualmente, também fazemos o perfilamento dos dados, ou seja, analisar suas distribuições e possíveis imperfeições de nossas variáveis. Para isso, o **Databricks** já oferece uma funcionalidade pronta para acelerar o processo.

Na célula com os dados da tabela **visits**, execute os seguintes passos:
- Clique no ícone **+**, logo abaixo do código e ao lado de **Table**
- Clique em **Data Profile**

Agora, podemos analisar os dados para identificar variáveis que possam ser interessantes **ou não** para o nosso modelo. Além de ações que possam ser necessárias para aumentar seu poder preditivo.

## Exercício 01.02 - Preparação dos Dados

Com a exploração concluída, já temos uma série de insumos para melhorar nosso conjunto de dados.

Vamos começar a prepará-lo!

### A. Dados de cliente

Nossos dados só contém a data de subscrição do cliente, porém essa informação não agrega nenhum poder preditivo ao nosso modelo. Então, vamos calcular o customer_lifetime.

Selecione a célula abaixo e use **Ctrl+I** para solicitar ao **Databricks Assistant**:

`Usando o customer_df, crie uma coluna customer_lifetime com a diferença entre o customer_since_date e hoje`

In [0]:
from pyspark.sql.functions import col, datediff, current_date

df_customers = df_customers.withColumn('customer_lifetime', datediff(current_date(), col('customer_since_date')))
display(df_customers)

Adicionalmente, vamos remover colunas que não carregam informação. Para isso, solicite:

`Remova as colunas card_number, first_name, last_name, customer_since_date`

In [0]:
columns_to_drop = ['card_number', 'first_name', 'last_name', 'customer_since_date']
df_customers = df_customers.drop(*columns_to_drop)
display(df_customers)

### B. Dados de localização

Nossos dados contém uma coluna estruturada que agrega uma série de informações de localização. Como, em geral, o zip (CEP) fornece um maior poder preditivo, vamos extrair essa informação.

Faça a solicitação abaixo:
`Usando o df_locations, extraia o campo zip do struct city_state_zip`

In [0]:
df_locations = df_locations.withColumn('zip', col('city_state_zip.zip'))
display(df_locations)

Adicionalmente, vamos remover colunas que não carregam informação. Para isso, solicite:

`Remova a coluna city_state_zip`

In [0]:
df_locations = df_locations.drop('city_state_zip')
display(df_locations)

### C. Combinando os dados

Faça a solicitação abaixo para o **Databricks Assistant**:

`Faça um join entre as tabelas visits, customers e locations`

In [0]:
df_joined = df_visits.join(df_customers, on='customer_id', how='inner') \
                     .join(df_locations, df_visits.atm_id == df_locations.atm_id, how='inner')
display(df_joined)

Agora, vamos remover os IDs e outras colunas que não carregam informação:
  
`Remova todos os ids do dataframe. Também remova as colunas year, month, day, min e sec`

In [0]:
columns_to_drop = [col for col in df_joined.columns if 'id' in col or col in ['year', 'month', 'day', 'min', 'sec']]
df_joined = df_joined.drop(*columns_to_drop)
display(df_joined)

### D. Tratando problemas das variáveis

Para melhorar o poder preditivo do zip, vamos fazer um *encoding por probabilidade*, ou seja, substituir seu valor categórico pela sua probabilidade de fraude. Para isso, solicite:

`Usando o dataframe anterior, calcule o percentual de fraud_report = 1 para cada zip. Adicione essa informação ao dataframe, criando a coluna zip_enc. Remova a coluna zip`

In [0]:
from pyspark.sql import functions as F

# Calculate the percentage of 1 in fraud_report for each zip
fraud_percentage = df_joined.groupBy('zip') \
                            .agg((F.sum(F.col('fraud_report')) / F.count('*')).alias('zip_enc'))

# Join the calculated percentage back to the original dataframe
df_joined = df_joined.join(fraud_percentage, on='zip', how='left')

# Drop the zip column
df_joined = df_joined.drop('zip')

display(df_joined)

### E. Salvando tabela final

Com os dados preparados, vamos salvá-lo em uma nova tabela para começarmos a treinar nossos modelos com o **AutoML**.

Solicite:

`Salve os dados na tabela fraud_abt`

In [0]:
df_joined.write.mode('overwrite').saveAsTable("fraud_abt")

**Parabéns!**

Você concluiu a exploração e preparação dos dados. Com isso, nossos modelos poderão alcançar performances mais elevadas, transferindo nosso conhecimento de negócio para os dados, além de reduzir o tempo e, consequentemente, os custos para o treinamento destes modelos!